In [16]:
import pandas as pd


df = pd.read_csv("outputs/spreads.csv")


df["formatted_date"] = pd.to_datetime(df["Date"]).dt.strftime("on %B %-d, %Y")
df.rename(columns={"Ticker Pair": "Ticker_Pair"}, inplace=True)


df[["tick1", "tick2"]] = df["Ticker_Pair"].str.split("-", expand=True)


df["chronobert_text"] = (
    df["formatted_date"] + ", the spread between " +
    df["tick1"] + " and " + df["tick2"] +
    " was " + df["Spread"].astype(str) + "."
)


df.drop(columns=["formatted_date", "tick1", "tick2"], inplace=True)
df.head()

,Date,Ticker_Pair,Spread,Return,chronobert_text
0,2019-02-28,AMD-NVDA,-1.319545,NaN,"on February 28, 2019, the spread between AMD a..."
1,2019-03-31,AMD-NVDA,-2.016619,-0.079436,"on March 31, 2019, the spread between AMD and ..."
2,2019-04-30,AMD-NVDA,-1.311970,0.074660,"on April 30, 2019, the spread between AMD and ..."
3,2019-05-31,AMD-NVDA,1.456129,0.242786,"on May 31, 2019, the spread between AMD and NV..."
4,2019-06-30,AMD-NVDA,0.568122,-0.104398,"on June 30, 2019, the spread between AMD and N..."


In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="manelalab/chrono-bert-v1-19991231",
    tokenizer="manelalab/chrono-bert-v1-19991231"
)

text = ""
for sentence in df["chronobert_text"]:
    text += sentence + " "

results = fill_mask(text)

for r in results:
    print(f"{r['sequence']} — score: {r['score']:.4f}")
